In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class NeuralNetworkXOR:
    def __init__(self, x_mau, y_mau, num_input_features, num_hidden_nodes, num_output_nodes, num_layers, max_iterations, lr):
        self.x_mau = x_mau
        self.y_mau = y_mau
        self.num_input_features = num_input_features
        self.num_hidden_nodes = num_hidden_nodes
        self.num_output_nodes = num_output_nodes
        self.num_layers = num_layers
        self.max_iterations = max_iterations
        self.lr = lr
        self.w = [None] * (self.num_layers + 1)
        self.n = [None] * (self.num_layers + 1)
        self.H = [None] * (self.num_layers + 1)
        self.layer_inputs_with_bias = [None] * (self.num_layers + 1)
        self.delta = [None] * (self.num_layers + 1)
        self.dw = [None] * (self.num_layers + 1)        
        self._initialize_weights()

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, n):
        s = self.sigmoid(n)
        return s * (1 - s)

    def _initialize_weights(self):
        np.random.seed(0)
        # Lớp 1 (lớp ẩn đầu tiên hoặc lớp output nếu num_layers=1)
        if self.num_layers == 1:
            self.w[1] = np.random.uniform(-1, 1, (self.num_output_nodes, self.num_input_features + 1))
        else:
            self.w[1] = np.random.uniform(-1, 1, (self.num_hidden_nodes, self.num_input_features + 1))
        for i in range(2, self.num_layers + 1):
            if i == self.num_layers:  # Lớp output
                self.w[i] = np.random.uniform(-1, 1, (self.num_output_nodes, self.num_hidden_nodes + 1))
            else:  # Lớp ẩn khác
                self.w[i] = np.random.uniform(-1, 1, (self.num_hidden_nodes, self.num_hidden_nodes + 1))

    def _forward_pass(self, x_in, N_samples):
        for i in range(1, self.num_layers + 1):
            if i == 1:
                current_layer_input_actual = x_in
            else:
                prev_layer_H_output = self.H[i - 1]
                bias_row = np.ones((1, N_samples))
                current_layer_input_actual = np.vstack((bias_row, prev_layer_H_output))           
            self.layer_inputs_with_bias[i] = current_layer_input_actual          
            self.n[i] = np.dot(self.w[i], current_layer_input_actual)
            self.H[i] = self.sigmoid(self.n[i])       
        return self.H[self.num_layers]

    def _compute_loss(self, predict, y_true):
        epsilon = 1e-8
        J_cost = -(y_true * np.log(predict + epsilon) + (1 - y_true) * np.log(1 - predict + epsilon))
        return np.mean(J_cost)

    def _backpropagation(self, y_true, N_samples):
        for i in range(self.num_layers, 0, -1):
            if i == self.num_layers:
                self.delta[i] = (self.H[i] - y_true) * self.sigmoid_derivative(self.n[i])
            else:
                weights_from_next_layer_no_bias = self.w[i + 1][:, 1:]
                error_propagated_sum = np.dot(weights_from_next_layer_no_bias.T, self.delta[i + 1])
                self.delta[i] = error_propagated_sum * self.sigmoid_derivative(self.n[i])           
            self.dw[i] = np.dot(self.delta[i], self.layer_inputs_with_bias[i].T) / N_samples

    def _update_weights(self):
        for i in range(1, self.num_layers + 1):
            self.w[i] -= self.lr * self.dw[i]

    def train(self):
        N = self.x_mau.shape[0]
        ones_column = np.ones((N, 1))
        x_in = np.hstack((ones_column, self.x_mau)).T
        y_true = self.y_mau.T
        loss_history = []
        for epoch in range(self.max_iterations):
            # forward pass
            predict = self._forward_pass(x_in, N)          
            # Compute loss
            current_loss = self._compute_loss(predict, y_true)
            loss_history.append(current_loss)
            if epoch % 1000 == 0 or epoch == self.max_iterations - 1:
                print(f"Epoch {epoch}, Loss: {current_loss:.4f}")
            # Backpropagation
            self._backpropagation(y_true, N)           
            # Update weights
            self._update_weights()    
        return loss_history

    def predict(self):
        N = self.x_mau.shape[0]
        ones_column = np.ones((N, 1))
        x_in = np.hstack((ones_column, self.x_mau)).T
        # Chỉ cần lan truyền xuôi để dự đoán
        predict = self._forward_pass(x_in, N).T 
        final_predictions = np.round(predict)      
        return predict, final_predictions
    
x_mau = np.array([
    [0, 0],
    [1, 0],
    [0, 1],
    [1, 1]
])
y_mau = np.array([[0], [1], [1], [0]])

num_samples = x_mau.shape[0]
num_input_features = x_mau.shape[1]
num_output_nodes = y_mau.shape[1]

nn_xor = NeuralNetworkXOR(
    x_mau,
    y_mau,
    num_input_features=num_input_features,
    num_hidden_nodes=3, # Số nơ-ron ẩn
    num_output_nodes=num_output_nodes,
    num_layers=2,   # Số tầng ẩn
    max_iterations=10000,
    lr=0.1
)
loss_history = nn_xor.train()
output_probs, predictions = nn_xor.predict()
for i in range(x_mau.shape[0]):
    print(f"Input: {x_mau[i]}, Expected: {y_mau[i][0]}, Predicted: {int(predictions[i][0])}, Xác Suất: {output_probs[i][0]:.4f}")

# Vẽ đồ thị loss
plt.figure(figsize=(10, 5))
plt.plot(loss_history, label='Loss')
plt.title('Loss over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.yscale('log')
plt.show()